In [2]:
pip install jdatetime

In [4]:
import pandas as pd
import numpy as np
import jdatetime
from datetime import datetime, timedelta
import random

In [26]:
num_records = 1000
cities = ['Tehran', 'Isfahan', 'Shiraz', 'Mashhad', 'Tabriz']
vehicle_types = ['Motorcycle', 'Van', 'Truck', 'Airplane']
shipping_modes = ['Ground', 'Air']
start_date = datetime(2025, 1, 1)

data = []

In [27]:
for i in range(num_records):
  order_id = 1000 + i
  customer_id = f'C{str(random.randint(1, 200)).zfill(3)}'

  order_date_g = start_date + timedelta(days=random.randint(0, 30))
  ship_date_g = order_date_g + timedelta(days=random.choice([0, 1, 2])) #ShipDate: When the package leaves the warehouse.

  sla_days = 2 # Service Level Agreement days
  actual_delay = random.choices([0, 1, 2, 3, -1], weights=[60, 15, 10, 10, 5])[0]
  delivery_date_g = ship_date_g + timedelta(days=sla_days + actual_delay) # DeliveryDate: When it actually arrives.

  delay_days = (delivery_date_g - ship_date_g).days - sla_days
  if delay_days < 0:
    status = 'Early'
  elif delay_days == 0:
    status = 'On Time'
  else:
    status = 'Delayed'

  # to persian date
  order_date_j = jdatetime.date.fromgregorian(date=order_date_g.date())
  ship_date_j = jdatetime.date.fromgregorian(date=ship_date_g.date())
  delivery_date_j = jdatetime.date.fromgregorian(date=delivery_date_g.date())

  origin = random.choice(cities)
  destination = random.choice(cities)
  if origin == destination:
    distance_km = round(np.random.normal(loc=40, scale=12))
    vehicle_ = vehicle_types.copy()
    vehicle_.remove('Airplane')
    vehicle = random.choice(vehicle_)
  else:
    distance_km = round(np.random.normal(loc=300, scale=100))
    vehicle = random.choice(vehicle_types)
  data.append([
        order_id, customer_id, str(order_date_j), str(ship_date_j), str(delivery_date_j),
        origin, destination, vehicle, status, delay_days, distance_km
    ])


In [28]:
columns = [
    'OrderID',
    'CustomerID',
    'OrderDate',
    'ShipDate',
    'DeliveryDate',
    'OriginCity',
    'DestinationCity',
    'VehicleType',
    'DeliveryStatus',
    'DelayDays',
    'DistanceKM'
]

df = pd.DataFrame(data, columns=columns)

In [29]:
df.head()

,OrderID,CustomerID,OrderDate,ShipDate,DeliveryDate,OriginCity,DestinationCity,VehicleType,DeliveryStatus,DelayDays,DistanceKM
0,1000,C034,1403-10-22,1403-10-24,1403-10-26,Tehran,Tehran,Van,On Time,0,36
1,1001,C065,1403-10-20,1403-10-22,1403-10-24,Shiraz,Isfahan,Motorcycle,On Time,0,285
2,1002,C166,1403-11-09,1403-11-11,1403-11-13,Tehran,Tehran,Motorcycle,On Time,0,36
3,1003,C073,1403-10-29,1403-10-29,1403-11-02,Isfahan,Isfahan,Truck,Delayed,1,23
4,1004,C144,1403-10-17,1403-10-18,1403-10-23,Tehran,Tabriz,Van,Delayed,3,140


In [30]:
df.to_excel('Delivery Performance Data.xlsx', index=False)